In [12]:
'''
! pip uninstall torch torchvision torchaudio -y
! pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2

! pip install transformers
from transformers import CLIPProcessor, CLIPModel
print("Transformers library installed successfully!")

! pip uninstall scikit-learn
! pip install scikit-learn

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
print("scikit-learn version:", sklearn.__version__)

! pip install pillow
'''



  Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   - -------------------------------------- 1.3/44.8 MB 6.7 MB/s eta 0:00:07
   --- ------------------------------------ 3.7/44.8 MB 9.1 MB/s eta 0:00:05
   ------ --------------------------------- 6.8/44.8 MB 11.3 MB/s eta 0:00:04
   --------- ------------------------------ 10.5/44.8 MB 12.8 MB/s eta 0:00:03
   ------------- -------------------------- 15.5/44.8 MB 15.0 MB/s eta 0:00:02
   ------------------ --------------------- 20.7/44.8 MB 16.6 MB/s eta 0:00:02
   ----------------------- ---------------- 26.7/44.8 MB 18.2 MB/s eta 0:00:01
   ---------------------------- ----------- 32.0/44.8

In [12]:
import sys
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
import sklearn
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import models, transforms
from transformers import CLIPProcessor, CLIPModel
import os

# Define the path to your images
image_dir = 'E:/Oxford-pets/images'

# Load the Oxford Pets dataset from local directory
def load_dataset(image_dir):
    image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]
    return image_files

train_dataset = load_dataset(image_dir)

# Define image transformations for pre-processing
def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image)

# 1. Feature Extraction using ResNet-50
def extract_features_resnet(images):
    model = models.resnet50(pretrained=True)
    model.eval()
    features = []
    for img_path in tqdm(images):
        img = Image.open(img_path).convert("RGB")
        img_tensor = preprocess_image(img)
        with torch.no_grad():
            feature = model(img_tensor.unsqueeze(0)).numpy().flatten()
            features.append(feature)
    return np.array(features)

# 2. Feature Extraction using CLIP
def extract_features_clip(images, processor, model):
    features = []
    for img_path in tqdm(images):
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt")
        with torch.no_grad():
            feature = model.get_image_features(**inputs).numpy().flatten()
            features.append(feature)
    return np.array(features)

# Compare different techniques

def compare_techniques(query_img, images, feature_extractor, processor=None, model=None):
    if processor is None or model is None:
        # Feature extractor does not need processor and model
        # Ensure that query_img and images are file paths
        query_feature = feature_extractor([query_img])[0]
        image_features = feature_extractor(images)
    else:
        # Feature extractor needs processor and model
        # Ensure that query_img and images are PIL Images
        query_feature = feature_extractor([query_img], processor, model)[0]
        image_features = feature_extractor(images, processor, model)
    similarities = cosine_similarity([query_feature], image_features)
    return similarities[0]


# Load a sample image for querying
def load_sample_image(image_path):
    return Image.open(image_path)

if __name__ == "__main__":
    # Load sample images
    sample_images = [img_path for img_path in train_dataset[:10]]  # Taking file paths for ResNet-50

    # CLIP Processor and Model
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

    # Load and preprocess query image
    query_image_path = 'E:/Oxford-pets/archive/images/images/Abyssinian_48.jpg'
    query_image = Image.open(query_image_path).convert("RGB")

    # Extract features and compare
    print("Comparing using ResNet-50...")
    similarities_resnet = compare_techniques(query_image_path, sample_images, extract_features_resnet)

    print("Comparing using CLIP...")
    similarities_clip = compare_techniques(query_image, sample_images, extract_features_clip, clip_processor, clip_model)

    # Print top-5 similar images
    print("Top-5 most similar images using ResNet-50:")
    top5_resnet = np.argsort(similarities_resnet)[-5:]
    print(top5_resnet)

    print("Top-5 most similar images using CLIP:")
    top5_clip = np.argsort(similarities_clip)[-5:]
    print(top5_clip)



Comparing using ResNet-50...


e:\AI_Projects\Images_RIS\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\AI_Projects\Images_RIS\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Hafeez/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 21.3MB/s]
  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'Image' object has no attribute 'read'

Comparing using ResNet-50...


TypeError: extract_features_resnet() takes 1 positional argument but 3 were given